In [62]:
!pip install openpyxl torch pandas sentence_transformers bertopic nbformat

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 13.0 MB/s eta 0:00:00


In [47]:
import random
import torch
import pickle

import re

import pandas
pandas.set_option('display.max_rows', None)

from sentence_transformers import SentenceTransformer, models
from bertopic import BERTopic

## Data importing and cleaning

* Remove links, images, @usernames and RTs indicating re-tweeting.
* Remove all content which is empyt after this

In [48]:
data = pandas.read_csv("realDonaldTrump_in_office.csv")

## remove links and images
data['Tweet.Text'] = data['Tweet.Text'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
data['Tweet.Text'] = data['Tweet.Text'].replace(r'pic.twitter.com\S+', '', regex=True)

## remove usernames
data['Tweet.Text'] = data['Tweet.Text'].replace(r'@\S+', '', regex=True)

## remove RTs
data['Tweet.Text'] = data['Tweet.Text'].replace(r'RT', '', regex=True)

data['Tweet.Text'] = data['Tweet.Text'].str.strip()

## remove items which have no remaining content after cleaning it
data = data.drop( data[ data['Tweet.Text'] == '' ].index )

data.head()

,Unnamed: 0,ID,Time,Tweet.URL,Tweet.Text
0,1,@realDonaldTrump,2017-01-20 06:31,https://twitter.com/realDonaldTrump/status/82...,It all begins today! I will see you at 11:00 A...
1,2,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/82...,Today we are not merely transferring power fro...
2,3,@realDonaldTrump,2017-01-20 11:51,https://twitter.com/realDonaldTrump/status/82...,"power from Washington, D.C. and giving it back..."
3,4,@realDonaldTrump,2017-01-20 11:52,https://twitter.com/realDonaldTrump/status/82...,What truly matters is not which party controls...
4,5,@realDonaldTrump,2017-01-20 11:53,https://twitter.com/realDonaldTrump/status/82...,"January 20th 2017, will be remembered as the d..."


## BERTopics created

* Resetting seeds for each model, extracting the word embeddings and then running the analysis with as close to default settings as possible

In [49]:
def create_model( model_name ):

    torch.manual_seed(0)
    random.seed(0)

    word_embedding_model = models.Transformer(model_name, max_seq_length=256)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

    transformer_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    topic_model = BERTopic(embedding_model=transformer_model )
    topic_model.fit_transform(data['Tweet.Text'])

    reduced_topics = topic_model.reduce_topics(data['Tweet.Text'], nr_topics="auto")
    
    pickle.dump( ( reduced_topics, topic_model, transformer_model ) , file = open( f"{model_name.replace('/', '_').replace('..', '')}", "wb") )
    return reduced_topics, topic_model, transformer_model

In [50]:
capitalist, capitalist_full, capitalist_transformer = create_model("microsoft/deberta-v3-base")
marxist, marxist_full, marxist_transformer = create_model("../models/microsoft_deberta-v3-base-finetuned-masked-model")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

## Basic summaries

In [51]:
capitalist.merge_topics(data['Tweet.Text'], [-1, 0])
capitalist.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,18783,-1_the_to_and_of,"[the, to, and, of, in, is, for, that, on, are]","[Georgia, this is your chance to MAKE AMERICA ..."
1,0,808,0_great_big_on_enjoy,"[great, big, on, enjoy, is, news, will, we, no...","[Big Story, Big Win - Except in the Fake News,..."
2,1,219,1_true_great_wow_so,"[true, great, wow, so, thanks, job, book, cong...","[So true!, So true!, So true!]"
3,2,162,2_trump_says_flynn_via,"[trump, says, flynn, via, breaking, news, impe...",[Intel chief releases info on ‘unmasking’ of F...
4,3,95,3_kag2020_thank_you_draining,"[kag2020, thank, you, draining, lyin, swamp, j...","[THANK YOU FLORIDA! #KAG2020, THANK YOU! #KAG2..."
5,4,91,4_thank_you_great_nice,"[thank, you, great, nice, honor, so, true, job...","[Great news, thank you!, Great news, thank you..."
6,5,89,5_rating_approval_thank_party,"[rating, approval, thank, party, 95, republica...",[95% Approval Rating in the Republican Party. ...
7,6,75,6_interviewed_00_enjoy_tonight,"[interviewed, 00, enjoy, tonight, at, eastern,...",[I will be interviewed by tonight at 9:00 P.M...
8,7,74,7_great_usa_job_agree,"[great, usa, job, agree, wow, dirty, congratul...","[Great deal for USA! ‚Ä¶, I agree. Jovita will..."
9,8,74,8_obamagate_read_transcripts_transcript,"[obamagate, read, transcripts, transcript, fak...","[Read the transcripts!, Read the Transcripts!,..."


In [52]:
marxist.merge_topics(data['Tweet.Text'], [-1, 0])
marxist.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,18646,-1_the_to_and_of,"[the, to, and, of, is, in, for, that, on, are]",[The Trump Administration has accomplished mor...
1,0,324,0_via_trump_fbi_biden,"[via, trump, fbi, biden, election, says, clint...",[Sen. Ron Johnson Calls For FBI To Address Rep...
2,1,208,1_unemployment_000_jobs_since,"[unemployment, 000, jobs, since, lowest, rate,...","[Unemployment is down to 4.1%, lowest in 17 ye..."
3,2,202,2_see_soon_at_way,"[see, soon, at, way, everyone, you, tomorrow, ...","[On my way, see you soon!, On my way to the Gr..."
4,3,166,3_thank_maga_kag2020_you,"[thank, maga, kag2020, you, pennsylvania, kag,...","[Thank you! #MAGA, Thank you #MAGA, THANK YOU..."
5,4,148,4_gt_äù_maga_trump,"[gt, äù, maga, trump, trump2020, birthday, pre...",[Our aircrew has just landed from another miss...
6,5,111,5_00_interviewed_enjoy_eastern,"[00, interviewed, enjoy, eastern, at, conferen...",[Will be interviewed by tonight at 9:00 P.M. ...
7,6,87,6_true_wow_great_yes,"[true, wow, great, yes, interesting, amazing, ...","[True!, True!, True!]"
8,7,85,7_thank_you_nice_true,"[thank, you, nice, true, so, thanks, matt, bra...","[Thank you So true!, Thank you President TRUM..."
9,8,81,8_again_make_america_great,"[again, make, america, great, keep, making, pr...","[MAKE AMERICA GREAT AGAIN!, MAKE AMERICA GREAT..."


In [53]:
## Export to CSV

marxist.get_topic_info().to_excel("marxist.xlsx")
capitalist.get_topic_info().to_excel("capitalist.xlsx")

## Move to analytics

In [63]:
capitalist.visualize_barchart( top_n_topics= 12 ).write_image("capitalist-barchart.pdf")
capitalist.visualize_barchart( top_n_topics= 12 )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.013962439387820498, 0.015871171678970418,
                    0.01860176166415458, 0.020210111857061286,
                    0.024071428130989017],
              'xaxis': 'x',
              'y': [is  , enjoy  , on  , big  , great  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.12274514632859229, 0.1293650728040584, 0.14777104783823297,
                    0.22110523065894255, 0.30677417419646275],
              'xaxis': 'x2',
              'y': [thanks  , so  , wow  , great  , true  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.024179045476103368, 0.024894899552284327,
                    0.025360122696456707, 0.0297637745557847, 0.04432911924176812],
              'xaxis': 'x3',
              'y': [breaking  , via  , flynn  , says  , trump  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.08977574562995845, 0.12440796057993653, 0.12698001881537638,
                    0.17737908394020616, 0.32620798767921383],
              'xaxis': 'x4',
              'y': [lyin  , draining  , you  , thank  , kag2020  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06598472433037275, 0.06970224652283885, 0.10402191608644981,
                    0.177906135310815, 0.25481851280437395],
              'xaxis': 'x5',
              'y': [honor  , nice  , great  , you  , thank  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.12583466790901332, 0.12788223328237133, 0.13267663496696244,
                    0.16641909533798027, 0.18084309478353078],
              'xaxis': 'x6',
              'y': [95  , party  , thank  , approval  , rating  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.15993729807464044, 0.17634939161141316, 0.2805017241704399,
                    0.32269676245594114, 0.3442475720127474],
              'xaxis': 'x7',
              'y': [at  , tonight  , enjoy  , 00  , interviewed  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04698059608562148, 0.05214634985447615, 0.05411186199040091,
                    0.05504738915812705, 0.10118307096616225],
              'xaxis': 'x8',
              'y': [wow  , agree  , job  , usa  , great  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.1527403281385508, 0.16119721745099455, 0.2163140015961944,
                    0.28714663944810553, 0.33340865409400233],
              'xaxis': 'x9',
              'y': [fake  , transcript  , transcripts  , read  , obamagate  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0508198379969407, 0.33301410644700125, 0.5562127391066678,
                    0.6260348146370884, 0.6925960959094851],
              'xaxis': 'x10',
              'y': [making  , great  , america  , make  , again  ],
              'yaxis': 'y10'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04991425804185663, 0.06194544571844006, 0.08079

In [64]:
marxist.visualize_barchart( top_n_topics= 12 ).write_image("marxist-barchart.pdf")
marxist.visualize_barchart( top_n_topics= 12 )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.012476038219485742, 0.013810058639186216,
                    0.013848447935374262, 0.020974354457731503,
                    0.036152356120933524],
              'xaxis': 'x',
              'y': [election  , biden  , fbi  , trump  , via  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.030233419858904435, 0.034320391025655386,
                    0.03912962371910211, 0.04059744635348811, 0.05538761212572473],
              'xaxis': 'x2',
              'y': [lowest  , since  , jobs  , 000  , unemployment  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02869681509952937, 0.02871901651114885, 0.0345673037714878,
                    0.052958723877993844, 0.05750221337498036],
              'xaxis': 'x3',
              'y': [everyone  , way  , at  , soon  , see  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0553269689888752, 0.19815869792934862, 0.2012158857446761,
                    0.31325474541786785, 0.3206292208953887],
              'xaxis': 'x4',
              'y': [pennsylvania  , you  , kag2020  , maga  , thank  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.01757753016399493, 0.020088468596305836,
                    0.02213184590279919, 0.022244822393434272,
                    0.022846841165281677],
              'xaxis': 'x5',
              'y': [trump2020  , trump  , maga  , äù  , gt  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.13944803432887337, 0.14126934476513126, 0.18137758784433175,
                    0.23044821130139528, 0.24311246733491634],
              'xaxis': 'x6',
              'y': [at  , eastern  , enjoy  , interviewed  , 00  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.16571504043892665, 0.16686197523018692, 0.22794080931240668,
                    0.4528449881223808, 0.6365264807673396],
              'xaxis': 'x7',
              'y': [interesting  , yes  , great  , wow  , true  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.06519192583816785, 0.08012062264620189, 0.08866744036674448,
                    0.263448676213669, 0.4250567412852126],
              'xaxis': 'x8',
              'y': [so  , true  , nice  , you  , thank  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.07759314828411054, 0.28296100466367724, 0.49675337509418793,
                    0.5128543434600688, 0.5692474819398854],
              'xaxis': 'x9',
              'y': [keep  , great  , america  , make  , again  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.028532021136594133, 0.03038534839323916,
                    0.03143817716658775, 0.038557945083463115, 0.03999410251584481],
              'xaxis': 'x10',
              'y': [didn  , do  , this  , it  , they  ],
              'yaxis': 'y10'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.09509140901932862, 0.10654714

In [56]:
marxist.visualize_topics( title='Marxist intertopic distance map' ).write_image("marxist-intertopic-distance.pdf")
marxist.visualize_topics( title='Marxist intertopic distance map' )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'via | trump | fbi | biden | election', 324],
                                   [1, 'unemployment | 000 | jobs | since | lowest', 208],
                                   [2, 'see | soon | at | way | everyone', 202],
                                   ...,
                                   [70, 'rating | approval | party | republican | record', 11],
                                   [71, 'proclaims | donald | greek | day | 2017', 11],
                                   [72, 'greatest | hunt | witch | history | american', 11]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': array([324, 208, 202, 166, 148, 111,  87,  85,  81,  75,  73,  72,  66,  63,
                                         56,  44,  44,  39,  36,  35,  35,  33,  32,  29,  29,  29,  28,  27,
                                         26,  26,  26,  25,  25,  25,  24,  22,  22,  22,  20,  20,  20,  20,
                                         19,  19,  18,  18,  18,  17,  17,  17,  17,  17,  17,  17,  17,  16,
                                         16,  15,  14,  14,  13,  13,  13,  13,  12,  12,  12,  12,  11,  11,
                                         11,  11,  11]),
                         'sizemode': 'area',
                         'sizeref': 0.2025,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([  5.616782  ,   2.6646032 ,  -8.509525  ,  17.260284  ,   6.6805124 ,
                          -13.49347   , -10.193833  ,   0.9299329 ,  -9.219422  ,  -0.6176331 ,
                           -5.686418  ,   7.7769747 ,  17.470709  , -10.652859  ,  -0.5605305 ,
                          -10.406448  ,   0.6291151 ,  12.781894  ,   0.47144035,  -5.7546444 ,
                            6.4632616 ,  -9.099296  ,  19.662333  ,   7.591941  , -10.201287  ,
                          -10.780188  ,   0.9505562 ,   0.8101138 ,   2.701203  ,  19.687012  ,
                           -5.7290444 ,  16.052061  ,   8.545557  ,  19.44581   ,  -0.3599773 ,
                            5.698788  ,   7.8156753 ,  15.954957  , -13.493798  ,  -0.82351524,
                            1.1386728 ,  16.267868  ,  12.757149  ,  12.705263  ,   6.8398657 ,
                            5.2844524 ,  -8.707798  ,  16.10275   ,  17.13823   ,  15.843422  ,
                           -0.59032965,  -8.44236   ,   2.7771888 ,   6.5101595 ,   6.5659447 ,
                           17.500952  ,   7.4770446 ,   1.1182853 ,  19.38556   ,  -8.887985  ,
                           -8.677788  ,   5.3710403 ,   5.4724374 ,  -0.92998976,   7.6962047 ,
                           -0.52884674,  -8.376003  ,   5.6354647 ,   6.5436068 ,   1.1182832 ,
                            8.545516  ,   5.3261986 ,  -8.317497  ], dtype=float32),
              'xaxis': 'x',
              'y': array([  2.1087854 ,  -9.270283  ,   5.4415174 ,  -0.03097821,   2.5915    ,
                          -12.907322  ,  16.415754  ,  10.598036  ,   4.6972733 ,  -7.215893  ,
                           -3.6437063 ,   1.7743675 ,  -0.24303289,  16.87386   ,  -6.736425  ,
                           16.627579  ,  10.8971405 ,  -3.766557  ,  11.053774  ,  -3.714726  ,
                            2.2933571 ,   4.7099986 ,  13.264559  ,   1.953959  ,  16.422419  ,
                           17.001194  ,  10.577447  ,  10.717829  ,  -9.307164  ,  13.289457  ,
                           -3.6832817 ,  10.486113  ,  -8.061158  ,  13.045875  ,  -6.7925577 ,
                            2.2703006 ,   1.7357521 ,  10.388799  , -12.907732  ,  -7.3659163 ,
            

In [57]:
capitalist.visualize_topics( title='Capitalist intertopic distance map' ).write_image("capitalist-intertopic-distance.pdf")
capitalist.visualize_topics( title='Capitalist intertopic distance map' )

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'great | big | on | enjoy | is', 808],
                                   [1, 'true | great | wow | so | thanks', 219],
                                   [2, 'trump | says | flynn | via | breaking', 162],
                                   [3, 'kag2020 | thank | you | draining | lyin', 95],
                                   [4, 'thank | you | great | nice | honor', 91],
                                   [5, 'rating | approval | thank | party | 95', 89],
                                   [6, 'interviewed | 00 | enjoy | tonight | at', 75],
                                   [7, 'great | usa | job | agree | wow', 74],
                                   [8, 'obamagate | read | transcripts | transcript | fake', 74],
                                   [9, 'again | make | america | great | making', 73],
                                   [10, 'promises | kept | makeamericagreatagain | jobs | gt', 66],
                                   [11, 'knew | bang | he | no | you', 54],
                                   [12, 'thank | you | much | so | very', 53],
                                   [13, 'thank | you | matt | mark | brad', 49],
                                   [14, 'is | and | the | he | lying', 48],
                                   [15, 'thank | maga | you | kag2020 | florida', 40],
                                   [16, 'task | briefing | force | press | live', 39],
                                   [17,
                                    'neverforget | september11th | usa | makeamericagreatagain | buyamericanhireamerican',
                                    38],
                                   [18, 'says | trump | anti | mcgowan | scandal', 38],
                                   [19, 'thank | you | much | so | ', 37],
                                   [20, 'great | guy | book | congratulations | this', 34],
                                   [21, 'holds | conference | live | news | president', 27],
                                   [22, 'thank | you | birthday | franklin | harry', 27],
                                   [23, 'and | button | no | lebron | his', 27],
                                   [24, 'happy | thanksgiving | merry | hunt | witch', 27],
                                   [25, 'withheld | zelensky | ralston | dugan | cmt', 26],
                                   [26, 'maga | thank | you | rep | let', 26],
                                   [27, 'prime | minister | leader | united | bilateral', 26],
                                   [28, 'thank | michigan | you | pennsylvania | liberate', 25],
                                   [29, 'congratulations | haha | karma | thanks | thx', 25],
                                   [30, 'harassment | presidential | distancing | social | warrantless',
                                    25],
                                   [31, 'join | tickets | 00pm | at | me', 25],
                                   [32, 'conference | eastern | white | 30 | house', 24],
                                   [33, 'diary | convenient | remember | appeal | admission', 24],
                                   [34, 'thank | you | mike | narendramodi | ritchie', 23],
                                   [35, 'jobs | vote | winning | maga | usa', 21],
                                   [36, 'vows | cbo | 83 | foxnews | ms', 20],
                                   [37, 'yikes | jupiter | honor | 1000 | stunning', 20],
                                   [38, 'de | la | en | potusinjapan | día', 19],
                                   [39, 'together | again | america | making | great', 19],
                                   [40, 'love | you | thank | minnesota | west', 19],
                                   [41, 'thank | you | florida | pennsylvania | ohio', 18],
                                   [42, 'bless | god | maga | exoneration | patrick', 18],
                                   [43, 'order | law | amp |  | ', 18],
        

## Focus on the a few identified differences

In [68]:
marxist.get_representative_docs(62)


['DOW, S&amp;P 500 and NASDAQ close at record highs! #MAGA',
 '“DOW, NASDAQ, S&amp;P 500 CLOSE AT RECORD HIGHS”',
 '“DOW, NASDAQ, S&amp;P 500 CLOSE AT RECORD HIGHS”']

In [59]:
marxist.get_representative_docs(47)

['NEWS! Johnson &amp; Johnson Begins Final-Stage Testing of SINGLE-SHOT Covid-19 Vaccine. The 4th Phase III trial for a Covid vaccine in the US.',
 'Hurricane #Dorian Advisory 40A: Eye of Dorian Beginning to Inch Northwestward.',
 'Hurricane #Dorian Advisory 36: Category 5 Dorian Continues to Have Devastating Impacts On Grand Bahama Island.']

In [69]:
capitalist.transform( marxist.get_representative_docs(62) )

([-1, -1, -1], array([0., 0., 0.]))

In [67]:
capitalist.get_representative_docs( 24 )

['HAPPY THANKSGIVING!', 'HAPPY THANKSGIVING!', 'HAPPY THANKSGIVING!']

In [ ]:
capitalist.transform( marxist.get_representative_docs(57) )